In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Atherosclerosis"
cohort = "GSE133601"

# Input paths
in_trait_dir = "../../input/GEO/Atherosclerosis"
in_cohort_dir = "../../input/GEO/Atherosclerosis/GSE133601"

# Output paths
out_data_file = "../../output/preprocess/Atherosclerosis/GSE133601.csv"
out_gene_data_file = "../../output/preprocess/Atherosclerosis/gene_data/GSE133601.csv"
out_clinical_data_file = "../../output/preprocess/Atherosclerosis/clinical_data/GSE133601.csv"
json_path = "../../output/preprocess/Atherosclerosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptional survey of peripheral blood links lower oxygen saturation during sleep with reduced expressions of CD1D and RAB20 that is reversed by CPAP therapy"
!Series_summary	"Sleep Disordered Breathing (SDB) is associated with a wide range of physiological changes, likely due in part to the influence of hypoxemia during sleep on gene expression. We studied gene expression in peripheral blood mononuclear cells in association with three measures of SDB: the Apnea Hypopnea Index (AHI); average oxyhemoglobin saturation (avgO2) during sleep; and minimum oxyhemoglobin saturation (minO2) during sleep. We performed discovery analysis in two community-based studies: the Multi-Ethnic Study of Atherosclerosis (MESA; N = 580) and the Framingham Offspring Study (FOS; N=571). Associations with false discovery rate (FDR) q-value<0.05 in one study were considered to have replicated if a p-value<0.05 was observed in the other study. Associations that replicat

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset is about gene expression in peripheral blood
# associated with SDB (Sleep Disordered Breathing). The study explicitly mentions transcriptional survey
# and gene expression analysis.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# Looking at the sample characteristics dictionary:
# - No explicit atherosclerosis trait information is directly visible
# - No age information is visible
# - No gender information is visible

# From background information, this study is about Sleep Disordered Breathing (SDB) rather than Atherosclerosis
# However, it mentions "Multi-Ethnic Study of Atherosclerosis (MESA)" as one of the source studies
# This is not a direct measurement of atherosclerosis in the current dataset

# There doesn't appear to be trait information directly related to atherosclerosis in this dataset
trait_row = None

# No age information
age_row = None

# No gender information
gender_row = None

# Define conversion functions even though they won't be used in this case
def convert_trait(value):
    if value is None:
        return None
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    # Convert to binary based on presence of atherosclerosis
    return None  # No conversion rule needed since trait is not available

def convert_age(value):
    if value is None:
        return None
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    # Try to convert to float for continuous age
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    if value is None:
        return None
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    # Convert to binary (0 for female, 1 for male)
    if value in ["female", "f"]:
        return 0
    elif value in ["male", "m"]:
        return 1
    return None

# 3. Save Metadata
# Determine trait availability
is_trait_available = trait_row is not None

# Validate and save cohort information (initial filtering)
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip this substep


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Atherosclerosis/GSE133601/GSE133601_series_matrix.txt.gz
Gene data shape: (19684, 30)
First 20 gene/probe identifiers:
Index(['100009676_at', '10000_at', '10001_at', '10002_at', '100033413_at',
       '100033414_at', '100033416_at', '100033418_at', '100033420_at',
       '100033422_at', '100033423_at', '100033424_at', '100033425_at',
       '100033426_at', '100033427_at', '100033428_at', '100033430_at',
       '100033431_at', '100033432_at', '100033433_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examine the gene identifiers from the preview
# The identifiers like '100009676_at', '10000_at' are probe IDs from microarray platforms
# These are not standard human gene symbols and will need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Preview the gene annotation dataframe
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Since we have identified that the Description column contains full gene names,
# let's use this for our mapping instead of trying to extract symbols
print("\nFull gene name examples from Description column:")
print(gene_annotation['Description'].head(10).tolist())

# Create mapping data using ID and Description (as Gene)
mapping_data = gene_annotation[['ID', 'Description']].copy()
mapping_data.rename(columns={'Description': 'Gene'}, inplace=True)
mapping_data = mapping_data.dropna(subset=['Gene'])  # Remove rows with missing gene names

# Check mapping data
print(f"\nMapping data shape: {mapping_data.shape}")
print(preview_df(mapping_data, n=5))

# Check for any empty gene descriptions
empty_gene_count = (mapping_data['Gene'].str.strip() == '').sum()
print(f"\nNumber of probes with empty gene descriptions: {empty_gene_count}")

# Filter out empty descriptions
mapping_data = mapping_data[mapping_data['Gene'].str.strip() != '']
print(f"Final mapping data shape after filtering: {mapping_data.shape}")
print(preview_df(mapping_data, n=5))

# Verify the mapping data covers a significant portion of the probes in gene_data
gene_data_probes = set(gene_data.index)
mapping_probes = set(mapping_data['ID'])
common_probes = gene_data_probes.intersection(mapping_probes)

print(f"\nProbes in gene data: {len(gene_data_probes)}")
print(f"Probes in mapping data: {len(mapping_probes)}")
print(f"Probes in both: {len(common_probes)} ({len(common_probes)/len(gene_data_probes):.2%} coverage)")



Gene annotation preview:
Columns in gene annotation: ['ID', 'SPOT_ID', 'Description']
{'ID': ['1_at', '10_at', '100_at', '1000_at', '10000_at'], 'SPOT_ID': ['1', '10', '100', '1000', '10000'], 'Description': ['alpha-1-B glycoprotein', 'N-acetyltransferase 2 (arylamine N-acetyltransferase)', 'adenosine deaminase', 'cadherin 2, type 1, N-cadherin (neuronal)', 'v-akt murine thymoma viral oncogene homolog 3 (protein kinase B, gamma)']}

Full gene name examples from Description column:
['alpha-1-B glycoprotein', 'N-acetyltransferase 2 (arylamine N-acetyltransferase)', 'adenosine deaminase', 'cadherin 2, type 1, N-cadherin (neuronal)', 'v-akt murine thymoma viral oncogene homolog 3 (protein kinase B, gamma)', 'uncharacterized LOC100009676', 'mediator complex subunit 6', 'nuclear receptor subfamily 2, group E, member 3', 'N-acetylated alpha-linked acidic dipeptidase 2', 'small nucleolar RNA, C/D box 116-1']

Mapping data shape: (19638, 2)
{'ID': ['1_at', '10_at', '100_at', '1000_at', '10000_

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns to use for mapping
# From previous step, we see the 'ID' column contains probe IDs matching gene_data's index
# The 'Description' column contains gene names, not symbols

# Since extract_human_gene_symbols isn't working well with this dataset's descriptions,
# we'll modify our approach to use the full descriptions

# 2. Create a mapping dataframe using probe IDs and full gene descriptions
mapping_data = gene_annotation[['ID', 'Description']].copy()
mapping_data = mapping_data.rename(columns={'Description': 'Gene'})
mapping_data = mapping_data.dropna(subset=['Gene'])
mapping_data = mapping_data[mapping_data['Gene'].str.strip() != '']

print(f"Gene mapping created with shape: {mapping_data.shape}")
print("Example of mapping (first 5 rows):")
print(preview_df(mapping_data, n=5))

# Check overlap with gene_data
gene_data_probes = set(gene_data.index)
mapping_probes = set(mapping_data['ID'])
common_probes = gene_data_probes.intersection(mapping_probes)
print(f"Probes in both gene data and mapping: {len(common_probes)} ({len(common_probes)/len(gene_data_probes):.2%} coverage)")

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression
# This handles the many-to-many relationship by:
# - Dividing probe values equally among mapped genes
# - Summing all contributions to each gene
try:
    gene_data = apply_gene_mapping(gene_data, mapping_data)
    
    print(f"Converted gene expression data shape: {gene_data.shape}")
    print("Example of gene expression data (first 5 genes):")
    print(preview_df(gene_data, n=5))

    # Normalize gene symbols to standardize and aggregate duplicate symbols
    gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Final gene expression data shape after normalization: {gene_data.shape}")
    print("Example of normalized gene expression data (first 5 genes):")
    print(preview_df(gene_data, n=5))

    # Save the processed gene expression data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error during gene mapping: {e}")
    
    # In case of failure, examine the mapping data more closely
    print("\nInvestigating mapping data structure:")
    print(f"Number of unique probe IDs in mapping: {mapping_data['ID'].nunique()}")
    print(f"Number of unique gene names in mapping: {mapping_data['Gene'].nunique()}")
    
    # Check for probe-gene relationships
    mapping_sample = mapping_data.sample(min(5, len(mapping_data)))
    print("\nSample of mapping data:")
    print(mapping_sample)


Gene mapping created with shape: (19638, 2)
Example of mapping (first 5 rows):
{'ID': ['1_at', '10_at', '100_at', '1000_at', '10000_at'], 'Gene': ['alpha-1-B glycoprotein', 'N-acetyltransferase 2 (arylamine N-acetyltransferase)', 'adenosine deaminase', 'cadherin 2, type 1, N-cadherin (neuronal)', 'v-akt murine thymoma viral oncogene homolog 3 (protein kinase B, gamma)']}
Probes in both gene data and mapping: 19638 (99.77% coverage)
Converted gene expression data shape: (2209, 30)
Example of gene expression data (first 5 genes):
{'GSM3912810': [44.64284666133333, 6.85290986, 10.75107161, 14.671153504, 5.728124292], 'GSM3912811': [44.31222287333333, 7.392122475, 10.7728531, 14.458826952, 6.08107333], 'GSM3912812': [44.4390841525, 7.258355125, 10.5879051, 13.966994956, 5.788481876], 'GSM3912813': [44.16653068816667, 7.124871182, 10.6206698, 14.001834505, 5.999150005], 'GSM3912814': [45.1576120165, 7.305069986, 10.37066336, 13.835656475, 5.84612428], 'GSM3912815': [45.128156772666664, 6.97

Gene expression data saved to ../../output/preprocess/Atherosclerosis/gene_data/GSE133601.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols using NCBI database
print("Normalizing gene symbols...")
gene_data = pd.read_csv(out_gene_data_file, index_col=0)
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")
print("First 10 normalized gene symbols:")
print(gene_data.index[:10])

# Save the normalized gene data
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to: {out_gene_data_file}")

# 2. We've determined in Step 2 that trait data is not available (trait_row was None)
# This is confirmed by the matrix data analysis
print("\nReconfirming clinical data availability...")
_, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
_, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Display clinical data features
print("Sample characteristics dictionary:")
sample_characteristics_dict = get_unique_values_by_row(clinical_data)
print(sample_characteristics_dict)

# We confirmed that this dataset doesn't contain Atherosclerosis trait data
is_trait_available = False

# 3. Since we don't have trait data, we cannot create valid linked data
# We'll report this in the cohort info and skip the remaining steps
note = "This GSE133601 dataset contains gene expression data related to Sleep Disordered Breathing (SDB) and CPAP therapy, but does not have direct measurements of Atherosclerosis. The study mentions Multi-Ethnic Study of Atherosclerosis (MESA) as a source study, but the current dataset focuses on oxygen saturation during sleep, not atherosclerosis."

# 4. Validate and save cohort info (initial assessment since we can't do final validation without trait data)
is_usable = validate_and_save_cohort_info(
    is_final=False,  # Using initial validation since we don't have trait data for final validation
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available  # Set to False based on our analysis
)

print(f"Dataset usability for {trait} study: {is_usable}")
if not is_usable:
    print(f"The {cohort} dataset does not contain {trait} measurements and cannot be used for this specific study.")

Normalizing gene symbols...


Gene data shape after normalization: (1276, 30)
First 10 normalized gene symbols:
Index(['A4GALT', 'AAA1', 'AAR2', 'AATK', 'ABCC11', 'ABCD1', 'ABCE1', 'ABI3',
       'ABO', 'ABRA'],
      dtype='object', name='Gene')
Normalized gene data saved to: ../../output/preprocess/Atherosclerosis/gene_data/GSE133601.csv

Reconfirming clinical data availability...
Sample characteristics dictionary:
{0: ['tissue: peripheral blood mononuclear cells'], 1: ['subject: 10031', 'subject: 11874', 'subject: 11992', 'subject: 30234', 'subject: 30665', 'subject: 30838', 'subject: 40044', 'subject: 40266', 'subject: 40657', 'subject: 11928', 'subject: 30031', 'subject: 40269', 'subject: 30624', 'subject: 40971', 'subject: 40197'], 2: ['timepoint: pre-CPAP', 'timepoint: post-CPAP']}
Dataset usability for Atherosclerosis study: False
The GSE133601 dataset does not contain Atherosclerosis measurements and cannot be used for this specific study.
